In [1]:

import os
import numpy as np
from tqdm import tqdm

from source.ConfigSettings import base_volume_unit_dim, base_voxel_size
from source.VolumeUnit import VolumeUnit
from source.AWMR_utils import pad_awmr_from_original, make_mesh, mesh_whole_block_singularize, split_into_resolution,pad_awmr_from_original, mesh_whole_block
from source.AWMRblock8x8 import AWMRblock8x8 as AWMRblock # TODO

from utils.TSDFDataset import TSDFDataset
from utils.evalUtils import customChamfer, customHausdorff, key_is_in

import open3d as o3d

import igl
import pickle
import pandas as pd

evaluate = True

dataset_name = 'soldier'
finest_voxel_size = 1.5
scale_factor = 0.002/finest_voxel_size
final_res = np.array([16,16,16])

volume_origin = np.load(f'../vunits/{dataset_name}/voxsize_{finest_voxel_size:.1f}/volume_origin_{finest_voxel_size:.1f}.npy')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


FileNotFoundError: [Errno 2] No such file or directory: '../vunits/soldier/voxsize_1.5/volume_origin_1.5.npy'

In [1]:
p1 = np.array([166.485504 ,144.843796 ,92.823845])
p2 = np.array([185.676086 ,187.746445 ,70.507355])
vunit_start = np.floor((p1 - np.squeeze(volume_origin))/(finest_voxel_size*base_volume_unit_dim))
vunit_end = np.floor((p2 - np.squeeze(volume_origin))/(finest_voxel_size*base_volume_unit_dim))

print(vunit_start, vunit_end)

NameError: name 'np' is not defined

In [47]:
import numpy as np

# (1, 4) 입력 벡터
input_vector = np.arange(12)
print(input_vector)
# 사용자 정의 가중치를 적용한 가중 평균 계산
# result = custom_weighted_average(input_vector)

nn = np.average([input_vector[0::4],input_vector[1::4],input_vector[2::4],input_vector[3::4]],axis=0, weights=[.1,.4,.4,.1])
nn2 = np.average([input_vector[0::4],input_vector[1::4],input_vector[2::4],input_vector[3::4]],axis=0, weights=[.25,.25,.25,.25])
print(nn)
print(nn2)

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[1.5 5.5 9.5]
[1.5 5.5 9.5]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  404k    0  404k    0     0  1187k      0 --:--:-- --:--:-- --:--:-- 1187k
